# Welcome to the Prognostics Model Library Tutorial

The goal of this notebook is to instruct users on how to use and extend the NASA PCoE Python Prognostics Model Package. 

First some background. The Prognostics Model Package is a python package for the modeling and simulation of the evolution of state for components, systems, and systems of systems, with a focus on simulating specific events. When used for prognostics, these events are typically system failures, such as a winding failure on a motor or full discharge of a battery. 

A few definitions:
* __Event__: Something that can be predicted (e.g., system failure, warning threshold). An event has either occurred or not. 
* __Event State__: Progress towards event occurring. Defined as a number where an event state of 0 indicates the event has occurred and 1 indicates no progress towards the event (i.e., fully healthy operation for a failure event). For gradually occurring events (e.g., discharge) the number will progress from 1 to 0 as the event nears. In prognostics, event state is frequently called "State of Health" or "SOH"
* __Inputs__: Control applied to the system being modeled (e.g., current drawn from a battery)
* __Outputs__: Measured sensor values from a system (e.g., voltage and temperature of a battery), outputs can be estimated from system state
* __States__: Internal parameters (typically hidden states) used to represent the state of the system- can be same as inputs/outputs but do not have to be. 

The `prog_models` package has the following structure
* `prog_models.PrognosticsModel` - parent class for all prognostics models - defines interfaces that a model must implement, and tools for simulating said model 
* `prog_models.models.*` - implemented models (e.g., pump, valve, battery)

In addition to the `prog_models` package, this repo includes many examples illustrating how to use the package (see `examples/`), a template for implementing a new model (`prog_model_template`), documentation (`https://nasa.github.io/prog_models`), and this tutorial (`tutorial.ipynb`).

Before you start, make sure to install prog_models using the following command:

    pip install prog_models

Now lets get started with some examples

## Using the included models

This first example is for using the included prognostics models. First thing to do is to import the model you would like to use:

In [ ]:
from prog_models.models import BatteryCircuit

This imports the BatteryCircuit model distributed with the `prog_models` package. See <https://nasa.github.io/prog_models/models.html> for details on this model, including supported configuration parameters.

Next, lets create a new battery using the default settings:

In [ ]:
batt = BatteryCircuit()

This creates a battery circuit model. You can also pass configuration parameters into the constructor as kwargs to configure the system, for example
### <center>`BatteryCircuit(qMax = 7856)`</center>
Alternatively, you can set the configuration of the system afterwards, like so:

In [ ]:
batt.parameters['qMax'] = 7856 

Lets use the model properties to check out some details of this model, first the model configuration:

In [ ]:
from pprint import pprint
print('Model configuration:')
pprint(batt.parameters)

You can save or load your model configuration using pickle

In [ ]:
import pickle
pickle.dump(batt.parameters, open('ex.pickle', 'wb'))

Then you can set your model configuration like below. This is useful for recording and restoring model configurations. Some users store model configuration as picked files to be restored from later.

In [ ]:
batt.parameters = pickle.load(open('ex.pickle', 'rb'))

Next, let's look at the inputs (loading) and outputs (measurements) for a BatteryCircuit model. These are the keys expected for inputs and outputs, respectively.

In [ ]:
print('inputs: ', batt.inputs)
print('outputs: ', batt.outputs)

Now lets look at what events we're predicting. This model only predicts one event, called EOD (End of Discharge), but that's not true for every model. See <https://nasa.github.io/prog_models/models.html>

In [ ]:
print('event(s): ', batt.events)

Finally, let's take a look at the internal states that the model uses to represent the system:

In [ ]:
print('states: ', batt.states)

### Simulate to Time
All those checks were just to take a look at the properties of this model. Now let's work towards simulating. 

Next, we define future loading. This is a function that describes how the system will be loaded as a function of time. Here we're defining a basic peasewise function.

In [ ]:
def future_loading(t, x=None):
    # Variable (piece-wise) future loading scheme 
    if (t < 600):
        i = 2
    elif (t < 900):
        i = 1
    elif (t < 1800):
        i = 4
    elif (t < 3000):
        i = 2
    else:
        i = 3
    return {'i': i}

At last it's time to simulate. First we're going to simulate forward 200 seconds. To do this we use the function simulate_to() to simulate to the specified time and print the results

In [ ]:
time_to_simulate_to = 200
sim_config = {
    'save_freq': 20, 
    'print': True  # Print states - Note: is much faster without
}
(times, inputs, states, outputs, event_states) = batt.simulate_to(time_to_simulate_to, future_loading, **sim_config)

We can also plot the results

In [ ]:
event_states.plot(ylabel= 'SOC')
outputs.plot(ylabel= {'v': "voltage (V)", 't': 'temperature (°C)'}, compact= False)

### Simulate To Threshold
Instead of specifying a specific amount of time, we can also simulate until a threshold has been met using the simulate_to_threshold() method

In [ ]:
options = { #configuration for this sim
    'save_freq': 100, # Frequency at which results are saved (s)
    'horizon': 5000 # Maximum time to simulate (s) - This is a cutoff. The simulation will end at this time, or when a threshold has been met, whichever is first
    }
(times, inputs, states, outputs, event_states) = batt.simulate_to_threshold(future_loading, **options)
event_states.plot(ylabel='SOC')
outputs.plot(ylabel= {'v': "voltage (V)", 't': 'temperature (°C)'}, compact= False)

Default is to simulate until any threshold is met, but we can also specify which event we are simulating to (any key from model.events) for multiple event models. 

### Adding Noise
There are two types of noise that can be added to a model in simulation, described below:
* __Process Noise__: Noise representing uncertainty in the model transition. Applied during state transition
* __Measurement Noise__: Noise representing uncertainty in the measurement process; e.g., sensor sensitivity, sensor misalignements, environmental effects. Applied during estimation of outputs from states. 

The amount of noise is considered a property of the model and can be set using the m.parameters['process_noise'] and m.parameters['measurement_noise'], respectively.

In this first example we will use the Thrown Object model (will be defined later) and set the process noise to zero.

In [ ]:
from prog_models.models import ThrownObject
m = ThrownObject(process_noise = 0)

def future_load(t=None, x=None):  # Cannot load thrown object (no inputs)
    return {}
event = 'impact'  # Simulate until impact

# Simulate to a threshold
(times, _, states, outputs, _) = m.simulate_to_threshold(future_load, threshold_keys=[event], dt=0.005, save_freq=1)
print('Example without noise')
print('\t- states: {}'.format(['{:.1f}s: {}'.format(t, x) for (t,x) in zip(times, states)])) 
print('\t- impact time: {}s'.format(times[-1]))

Now with this second example we apply process noise with a standard deviation of 0.5 to every state

In [ ]:
m = ThrownObject(process_noise = 0.5)

# Simulate to a threshold
(times, _, states, outputs, _) = m.simulate_to_threshold(future_load, threshold_keys=[event], dt=0.005, save_freq=1)
print('Example without noise')
print('\t- states: {}'.format(['{:.1f}s: {}'.format(t, x) for (t,x) in zip(times, states)])) 
print('\t- impact time: {}s'.format(times[-1]))

You can also specify different amounts of noise on different states, for example:

In [ ]:
m = ThrownObject(process_noise = {'x': 0.25, 'v': 0.75})

# Simulate to a threshold
(times, _, states, outputs, _) = m.simulate_to_threshold(future_load, threshold_keys=[event], dt=0.005, save_freq=1)
print('Example without noise')
print('\t- states: {}'.format(['{:.1f}s: {}'.format(t, x) for (t,x) in zip(times, states)])) 
print('\t- impact time: {}s'.format(times[-1]))

You can also define the shape of the noise to be uniform or triangular instead of normal. Finally, you can define your own function to apply noise for anything more complex. For more informaiton see examples.noise

## Building a new model
To build a model, create a seperate class to define the logic of the model. Do this by copying the file `prog_model_template.py` and replacing the functions with the logic specific to your model. 

For this example, we will model the throwing of an object directly into the air in a vacuum. This is not a particularly interesting problem, but it is simple and illustrates the basic functions of a PrognosticsModel.

The model is illustrated below:

In [ ]:
from prog_models import PrognosticsModel
from numpy import inf

class ThrownObject(PrognosticsModel):
    """
    Model that similates an object thrown directly into the air without air resistance
    """

    inputs = [] # no inputs, no way to control
    states = [
        'x', # Position (m) 
        'v'  # Velocity (m/s)
        ]
    outputs = [ # Anything we can measure
        'x' # Position (m)
    ]
    events = [ # Events that can/will occur during simulation
        'falling', # Event- object is falling
        'impact' # Event- object has impacted ground
    ]

    # The Default parameters for any ThrownObject. Overwritten by passing parameters into constructor as kwargs or by setting model.parameters
    default_parameters = {
        'thrower_height': 1.83, # m
        'throwing_speed': 40, # m/s
        'g': -9.81, # Acceleration due to gravity in m/s^2
        'process_noise': 0.0 # amount of noise in each step
    }    

    # First function: Initialize. This function is used to initialize the first state of the model.
    # In this case we do not need input (u) or output (z) to initialize the model, so we set them to None, but that's not true for every model.
    def initialize(self, u=None, z=None):
        self.max_x = 0.0
        return {
            'x': self.parameters['thrower_height'], # Thrown, so initial altitude is height of thrower
            'v': self.parameters['throwing_speed'] # Velocity at which the ball is thrown - this guy is an professional baseball pitcher
            }
    
    # Second function: state transition. 
    # State transition can be defined in one of two ways:
    #   1) Discrete models use next_state(x, u, dt) -> x'
    #   2) Continuous models (preferred) use dx(x, u) -> dx/dt
    #
    # In this case we choose the continuous model, so we define dx(x, u)
    # This function defines the first derivative of the state with respect to time, as a function of model configuration (self.parameters), state (x) and input (u).
    # Here input isn't used. But past state and configuration are.
    def dx(self, x, u):
        return {
            'x': x['v'],  # dx/dt = v
            'v': self.parameters['g'] # Acceleration of gravity
        }
    # Equivilantly, the state transition could have been defined as follows:
    # def next_state(self, x, u, dt):
    #     return {
    #         'x': x['x'] + x['v']*dt,
    #         'v': x['v'] + self.parameters['g']*dt
    #     }

    # Now, we define the output equation. 
    # This function estimates the output (i.e., measured values) given the system state (x) and system parameters (self.parameters).
    # In this example, we're saying that the state 'x' can be directly measured. 
    # But in most cases output will have to be calculated from state. 
    def output(self, x):
        return {
            'x': x['x']
        }

    # Next, we define the event state equation
    # This is the first equation that actually describes the progress of a system towards the events.
    # This function maps system state (x) and system parameters (self.parameters) to event state for each event.
    # Event state is defined as a number between 0 and 1 where 1 signifies no progress towards the event, and 0 signifies the event has occured.
    # The event keys were defined above (model.events)
    # Here the two event states are as follows:
    #  1) falling: 1 is defined as when the system is moving at the maximum speed (i.e., throwing_speed), and 0 is when velocity is negative (i.e., falling)
    #  2) impact: 1 is defined as the ratio of the current altitude (x) to the maximum altitude (max_x), and 0 is when the current altitude is 0 (i.e., impact)
    def event_state(self, x): 
        self.max_x = max(self.max_x, x['x']) # Maximum altitude
        return {
            'falling': max(x['v']/self.parameters['throwing_speed'],0), # Throwing speed is max speed
            'impact': max(x['x']/self.max_x,0) # Ratio of current altitude to maximum altitude
        }

    # Finally, we define the threshold equation.
    # This is the second equation that describes the progress of a system towards the events.
    # Note: This function is optional. If not defined, threshold_met will be defined as when the event state is 0.
    # However, this implementation is more efficient, so we included it
    # This function maps system state (x) and system parameters (self.parameters) a boolean indicating if the event has been met for each event.
    def threshold_met(self, x):
        return {
            'falling': x['v'] < 0,
            'impact': x['x'] <= 0
        }


Now the model can be generated and used like any of the other provided models

In [ ]:
m = ThrownObject()

def future_load(t, x=None):
        return {} # No loading
event = 'impact'  # Simulate until impact

(times, inputs, states, outputs, event_states) = m.simulate_to_threshold(future_load, threshold_keys=[event], dt=0.005, save_freq=1)

event_states.plot(ylabel= ['falling', 'impact'], compact= False)
states.plot(ylabel= {'x': "position (m)", 'v': 'velocity (m/s)'}, compact= False)

## Building a new model - advanced

### Derived Parameters 

Models can also include "derived parameters" (i.e., parameters that are derived from others). These can be set using the param_callbacks property. 

Let's extend the above model to include derived_parameters. Let's say that the throwing_speed was actually a function of thrower_height (i.e., a taller thrower would throw the ball faster). Here's how we would implement that

In [ ]:
# Step 1: Define a function for the relationship between thrower_height and throwing_speed.
def update_thrown_speed(params):
    return {
        'throwing_speed': params['thrower_height'] * 21.85
    }  # Assumes thrown_speed is linear function of height

# Step 2: Define the param callbacks
ThrownObject.param_callbacks = {
        'thrower_height': [update_thrown_speed]
    }  # Tell the derived callbacks feature to call this function when thrower_height changes.

Note: Usually we would define this menthod within the class. For this example, we're doing it separately to improve readability. Here's the feature in action

In [ ]:
obj = ThrownObject()
print("Default Settings:\n\tthrower_height: {}\n\tthowing_speed: {}".format(obj.parameters['thrower_height'], obj.parameters['throwing_speed']))

# Now let's change the thrower_height
print("changing height...")
obj.parameters['thrower_height'] = 1.75  # Our thrower is 1.75 m tall
print("\nUpdated Settings:\n\tthrower_height: {}\n\tthowing_speed: {}".format(obj.parameters['thrower_height'], obj.parameters['throwing_speed']))
print("Notice how speed changed automatically with height")

# Let's delete the callback so we can use the same model in the future:
ThrownObject.param_callbacks = {}

See also examples.new_model for more information on building models

### State Limits

In many cases, the values of the model states have certain physical limits. For example, temperature is limited by absolute zero. In these cases, it is useful to define those limits. In simulation, the defined limits are enforced as the state transitions to prevent the system from reaching an impossible state.

For example, the ThrownObject model can be extended as follows:

In [ ]:
ThrownObject.state_limits = {
        # object may not go below ground
        'x': (0, inf),

        # object may not exceed the speed of light
        'v': (-299792458, 299792458)
    }

Note: like derived parameters, these would typically be defined in class definition, not afterwards. They are defined afterwards in this case to illustrate the feature.

State limits can be applied directly using the apply_limits function. For example:

In [ ]:
x = {'x': -5, 'v': 3e8} # Too fast and below the ground
x = obj.apply_limits(x)
print(x)

Notice how the state was limited according to the model state limits and a warning was issued. The warning can be suppressed by suppressing ProgModelStateLimitWarning (`warnings.simplefilter('ignore', ProgModelStateLimitWarning)`)

See also examples.derived_params for more information on this feature.

## Parameter Estimation
Let's say we dont know the configuration of the above model. Instead, we have some data. We can use that data to estimate the parameters. 

First, we define the data:

In [ ]:
times = [0, 1, 2, 3, 4, 5, 6, 7, 8]
inputs = [{}]*9
outputs = [
    {'x': 1.83},
    {'x': 36.95},
    {'x': 62.36},
    {'x': 77.81},
    {'x': 83.45},
    {'x': 79.28},
    {'x': 65.3},
    {'x': 41.51},
    {'x': 7.91},
]

Next, we identify which parameters will be optimized

In [ ]:
keys = ['thrower_height', 'throwing_speed']

Let's say we have a first guess that the thrower's height is 20m, silly I know

In [ ]:
m.parameters['thrower_height'] = 20

Here's the state of our estimation with that assumption:

In [ ]:
print('Model configuration before')
for key in keys:
    print("-", key, m.parameters[key])
print(' Error: ', m.calc_error(times, inputs, outputs, dt=1e-4))

Wow, that's a large error. 

Let's run the parameter estimation to see if we can do better

In [ ]:
m.estimate_params([(times, inputs, outputs)], keys, dt=0.01)

# Print result
print('\nOptimized configuration')
for key in keys:
    print("-", key, m.parameters[key])
print(' Error: ', m.calc_error(times, inputs, outputs, dt=1e-4))

Much better!

See also examples.param_est for more details about this feature

## Conclusion
This is just the basics, there's much more to learn. Please see the documentation at <https://nasa.github.io/prog_models> and the examples in the `examples/` folder for more details on how to use the package, including:
* `examples.derived_params` : Example building models with derived parameters.
* `examples.state_limits`: Example building models with limits on state variables.
* `examples.param_est`: Example using the parameter estimation feature 
* `examples.dynamic_step_size`: Example simulating with dynamic (i.e., changing as a function of time or state) step size.
* `examples.events`: Example extending a model to include additional events, such as warning thresholds.
* `examples.benchmarking`: Example benchmarking the performance of a Prognostics Model
* `examples.future_loading`: Example with complex future loading schemes
* `examples.new_model`: Example building a new model
* `examples.noise`: Example demonstrating how noise can be added in simulation
* `examples.vectorized`: Example simulating a vectorized model
* `examples.sim`, `examples.sim_battery_eol`, `examples.sim_pump`, `examples.sim_valve`: Examples using specific models from `prog_models.models`

Thank you for trying out this tutorial. Open an issue on github (https://github.com/nasa/prog_models/issues) or email Chris Teubert (christopher.a.teubert@nasa.gov) with any questions or issues.